In [1]:
import h5py
import numpy as np
import os, sys
import pandas as pd
sys.path.append(os.path.abspath("../data/MillionSongSubset"))
import hdf5_getters
import tables

In [2]:
h5 = hdf5_getters.open_h5_file_read(os.path.join("../data/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5"))


In [8]:
num_songs = hdf5_getters.get_artist_playmeid(h5)
print(num_songs)

4479


218.93179


In [20]:
def print_structure(name, obj):
    print(name)

data.visititems(print_structure)

analysis
analysis/bars_confidence
analysis/bars_start
analysis/beats_confidence
analysis/beats_start
analysis/sections_confidence
analysis/sections_start
analysis/segments_confidence
analysis/segments_loudness_max
analysis/segments_loudness_max_time
analysis/segments_loudness_start
analysis/segments_pitches
analysis/segments_start
analysis/segments_timbre
analysis/songs
analysis/tatums_confidence
analysis/tatums_start
metadata
metadata/artist_terms
metadata/artist_terms_freq
metadata/artist_terms_weight
metadata/similar_artists
metadata/songs
musicbrainz
musicbrainz/artist_mbtags
musicbrainz/artist_mbtags_count
musicbrainz/songs


In [25]:
print(data["metadata"]["songs"][:10])

[(b'', 165270, 0.58179377, 0.40199754, b'ARD7TVE1187B99BFB1', nan, b'California - LA', nan, b'e77e51a5-4761-45b3-9847-2051f811e366', b'Casual', 4479, b'', 0, 0, b'Fear Itself', 300848, 0.60211999, b'SOMZWCG12A8C13C480', b"I Didn't Mean To", 3401791)]


In [12]:
import sqlite3
import pandas as pd
from pathlib import Path


In [13]:
# Update this path if your file is elsewhere
db_path = Path("../data/mxm_dataset.db")  # or Path("/Users/pushpita/Documents/ML Projects/echoagent/.../bow.db")

conn = sqlite3.connect(db_path)
print("Connected to:", db_path.resolve())


Connected to: /Users/pushpita/Documents/ML Projects/echoagent/data/mxm_dataset.db


In [14]:
# List tables
tables = pd.read_sql_query("""
SELECT name, type
FROM sqlite_master
WHERE type IN ('table', 'view')
ORDER BY name
""", conn)

tables

,name,type
0,lyrics,table
1,words,table


In [15]:
# Show schema for each table
for t in tables["name"]:
    print(f"\n=== {t} ===")
    display(pd.read_sql_query(f"PRAGMA table_info({t})", conn))


=== lyrics ===


,cid,name,type,notnull,dflt_value,pk
0,0,track_id,,0,None,0
1,1,mxm_tid,INT,0,None,0
2,2,word,TEXT,0,None,0
3,3,count,INT,0,None,0
4,4,is_test,INT,0,None,0



=== words ===


,cid,name,type,notnull,dflt_value,pk
0,0,word,TEXT,0,None,1


In [16]:
# Row counts for each table
counts = []
for t in tables["name"]:
    n = pd.read_sql_query(f"SELECT COUNT(*) AS n FROM {t}", conn).iloc[0]["n"]
    counts.append({"table": t, "rows": n})

pd.DataFrame(counts)


,table,rows
0,lyrics,19045332
1,words,5000


In [19]:
# Preview rows (works for the MXM sqlite built by mxm_dataset_to_db.py: words + lyrics)
for t in tables["name"]:
    print(f"\n=== Preview: {t} ===")
    display(pd.read_sql_query(f"SELECT * FROM {t} LIMIT 20", conn))



=== Preview: lyrics ===


,track_id,mxm_tid,word,count,is_test
0,TRAAAAV128F421A322,4623710,i,6,0
1,TRAAAAV128F421A322,4623710,the,4,0
2,TRAAAAV128F421A322,4623710,you,2,0
3,TRAAAAV128F421A322,4623710,to,2,0
4,TRAAAAV128F421A322,4623710,and,5,0
5,TRAAAAV128F421A322,4623710,a,3,0
6,TRAAAAV128F421A322,4623710,me,1,0
7,TRAAAAV128F421A322,4623710,it,1,0
8,TRAAAAV128F421A322,4623710,my,1,0
9,TRAAAAV128F421A322,4623710,is,2,0



=== Preview: words ===


,word
0,i
1,the
2,you
3,to
4,and
5,a
6,me
7,it
8,not
9,in


In [20]:
# How many unique tracks / words / lyric rows?
pd.read_sql_query("""
SELECT
  (SELECT COUNT(*) FROM words) AS n_words,
  (SELECT COUNT(*) FROM lyrics) AS n_lyrics_rows,
  (SELECT COUNT(DISTINCT track_id) FROM lyrics) AS n_tracks
""", conn)


,n_words,n_lyrics_rows,n_tracks
0,5000,19045332,237662


In [24]:
sys.path.append(os.path.abspath("../"))
from backend.data.embeddings import EmbeddingManager

mgr = EmbeddingManager()  # uses /Users/pushpita/Documents/ML Projects/echoagent/backend/data/chroma_db

print("track_text count:", mgr.track_collection.count())
print("lyrics count:", mgr.lyrics_collection.count())
print("peek:", mgr.track_collection.peek(limit=3))


E0000 00:00:1771858545.437294 6948981 instrument.cc:563] Metric with name 'grpc.resource_quota.calls_dropped' registered more than once. Ignoring later registration.
E0000 00:00:1771858545.437602 6948981 instrument.cc:563] Metric with name 'grpc.resource_quota.calls_rejected' registered more than once. Ignoring later registration.
E0000 00:00:1771858545.437605 6948981 instrument.cc:563] Metric with name 'grpc.resource_quota.connections_dropped' registered more than once. Ignoring later registration.
E0000 00:00:1771858545.437606 6948981 instrument.cc:563] Metric with name 'grpc.resource_quota.instantaneous_memory_pressure' registered more than once. Ignoring later registration.
E0000 00:00:1771858545.437609 6948981 instrument.cc:563] Metric with name 'grpc.resource_quota.memory_pressure_control_value' registered more than once. Ignoring later registration.
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry ev

track_text count: 100
lyrics count: 0
peek: {'ids': ['TRAAABD128F429CF47', 'TRAAAED128E0783FAB', 'TRAAAEF128F4273421'], 'embeddings': array([[-0.05046621, -0.09049937,  0.03075439, ...,  0.01763589,
        -0.01323299, -0.09263226],
       [-0.0031452 , -0.13569978,  0.01060263, ..., -0.01617149,
        -0.08005787, -0.08536361],
       [-0.05862552, -0.06566019,  0.07236075, ..., -0.05565469,
        -0.0189395 , -0.11603994]], shape=(3, 384)), 'documents': [None, None, None], 'uris': None, 'data': None, 'metadatas': [{'genre_source': 'msd_lastfm_map', 'genres_csv': 'oldies', 'has_bow': True, 'has_lyrics': False, 'lyrics_source': 'mxm', 'msd_track_id': 'TRAAABD128F429CF47', 'mxm_tid': '6477168', 'song_id': 'TRAAABD128F429CF47', 'tag_source': 'msd_lastfm_map', 'tags_csv': '60s|soul|pop|rock|malevocalists|oldies|1969', 'track_id': 'TRAAABD128F429CF47'}, {'genre_source': 'msd_lastfm_map', 'genres_csv': 'jazz', 'has_bow': True, 'has_lyrics': False, 'lyrics_source': 'mxm', 'msd_track_id'

In [25]:
results = mgr.query_track_embeddings("rock")
for r in results[:5]:
    print(r["id"], r["distance"], r["metadata"])

/Users/pushpita/anaconda3/envs/ml_pde/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


TRAADLH12903CA70EE 0.9400590658187866 {'genre_source': 'msd_lastfm_map', 'genres_csv': 'rock', 'has_bow': True, 'has_lyrics': False, 'lyrics_source': 'mxm', 'msd_track_id': 'TRAADLH12903CA70EE', 'mxm_tid': '1518335', 'song_id': 'TRAADLH12903CA70EE', 'tag_source': 'msd_lastfm_map', 'tags_csv': 'political|skapunk', 'track_id': 'TRAADLH12903CA70EE'}
TRAAGAM128F428DCBE 0.972769021987915 {'genre_source': 'msd_lastfm_map', 'genres_csv': 'rock', 'has_bow': True, 'has_lyrics': False, 'lyrics_source': 'mxm', 'msd_track_id': 'TRAAGAM128F428DCBE', 'mxm_tid': '4372024', 'song_id': 'TRAAGAM128F428DCBE', 'tag_source': 'msd_lastfm_map', 'tags_csv': 'punk|pune', 'track_id': 'TRAAGAM128F428DCBE'}
TRAAKAS128F4246013 1.0083093643188477 {'genre_source': 'msd_lastfm_map', 'genres_csv': 'rock', 'has_bow': True, 'has_lyrics': False, 'lyrics_source': 'mxm', 'msd_track_id': 'TRAAKAS128F4246013', 'mxm_tid': '1352626', 'song_id': 'TRAAKAS128F4246013', 'tag_source': 'msd_lastfm_map', 'tags_csv': 'progressiverock|

In [26]:
sample = mgr.track_collection.peek(limit=5)
sample["metadatas"]

Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


[{'genre_source': 'msd_lastfm_map',
  'genres_csv': 'oldies',
  'has_bow': True,
  'has_lyrics': False,
  'lyrics_source': 'mxm',
  'msd_track_id': 'TRAAABD128F429CF47',
  'mxm_tid': '6477168',
  'song_id': 'TRAAABD128F429CF47',
  'tag_source': 'msd_lastfm_map',
  'tags_csv': '60s|soul|pop|rock|malevocalists|oldies|1969',
  'track_id': 'TRAAABD128F429CF47'},
 {'genre_source': 'msd_lastfm_map',
  'genres_csv': 'jazz',
  'has_bow': True,
  'has_lyrics': False,
  'lyrics_source': 'mxm',
  'msd_track_id': 'TRAAAED128E0783FAB',
  'mxm_tid': '2516445',
  'song_id': 'TRAAAED128E0783FAB',
  'tag_source': 'msd_lastfm_map',
  'tags_csv': 'jazz|jamiecullum|singersongwriter|other|piano',
  'track_id': 'TRAAAED128E0783FAB'},
 {'genre_source': 'msd_lastfm_map',
  'genres_csv': 'rock',
  'has_bow': True,
  'has_lyrics': False,
  'lyrics_source': 'mxm',
  'msd_track_id': 'TRAAAEF128F4273421',
  'mxm_tid': '3759847',
  'song_id': 'TRAAAEF128F4273421',
  'tag_source': 'msd_lastfm_map',
  'tags_csv': 'ne